In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# import keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

#Import pretrained models
from tensorflow.keras.applications import VGG16

Monday : 
* EDA
* Modelling (ImageDataGenerator understanding, Custom CNN, start with Pretrained model

Questions:
- What the difference between resizing and cov ? 

To-do list :
- Helper function to loop the pretrained model and transfer it to Colab

Tuesday :

In [2]:
train_df=pd.read_pickle('./train_df.pkl')
train_df.head()

,ClassId,Path
0,Dangerous curve right,Train/20/00020_00000_00000.png
1,Dangerous curve right,Train/20/00020_00000_00001.png
2,Dangerous curve right,Train/20/00020_00000_00002.png
3,Dangerous curve right,Train/20/00020_00000_00003.png
4,Dangerous curve right,Train/20/00020_00000_00004.png


In [3]:
test_df=pd.read_pickle('./test_df.pkl')
test_df.head()

,ClassId,Path
0,Veh > 3.5 tons prohibited,Test/00000.png
1,Speed limit (30km/h),Test/00001.png
2,Keep right,Test/00002.png
3,Turn right ahead,Test/00003.png
4,Right-of-way at intersection,Test/00004.png


In [4]:
full_df=pd.read_pickle('./full_df.pkl')
full_df.head()

,ClassId,Path
0,20,Train/20/00020_00000_00000.png
1,20,Train/20/00020_00000_00001.png
2,20,Train/20/00020_00000_00002.png
3,20,Train/20/00020_00000_00003.png
4,20,Train/20/00020_00000_00004.png


In [ ]:
classes = {
    0: 'Speed limit (20km/h)',
    1: 'Speed limit (30km/h)',
    2: 'Speed limit (50km/h)',
    3: 'Speed limit (60km/h)',
    4: 'Speed limit (70km/h)',
    5: 'Speed limit (80km/h)',
    6: 'End of speed limit (80km/h)',
    7: 'Speed limit (100km/h)',
    8: 'Speed limit (120km/h)',
    9: 'No passing',
    10: 'No passing veh over 3.5 tons',
    11: 'Right-of-way at intersection',
    12: 'Priority road',
    13: 'Yield',
    14: 'Stop',
    15: 'No vehicles',
    16: 'Veh > 3.5 tons prohibited',
    17: 'No entry',
    18: 'General caution',
    19: 'Dangerous curve left',
    20: 'Dangerous curve right',
    21: 'Double curve',
    22: 'Bumpy road',
    23: 'Slippery road',
    24: 'Road narrows on the right',
    25: 'Road work',
    26: 'Traffic signals',
    27: 'Pedestrians',
    28: 'Children crossing',
    29: 'Bicycles crossing',
    30: 'Beware of ice/snow',
    31: 'Wild animals crossing',
    32: 'End speed + passing limits',
    33: 'Turn right ahead',
    34: 'Turn left ahead',
    35: 'Ahead only',
    36: 'Go straight or right',
    37: 'Go straight or left',
    38: 'Keep right',
    39: 'Keep left',
    40: 'Roundabout mandatory',
    41: 'End of no passing',
    42: 'End no passing veh > 3.5 tons'
}

l_classes = list(classes.values())
nb_classes = len(l_classes)

In [ ]:
width_im = 50
height_im = 50
image_size = (width_im, height_im)
batch_size = 64

In [ ]:
#Shuffle the rows of the training dataframe to be corretly split for validation
train_df = train_df.sample(frac=1).reset_index(drop=True)

#
train_datagen = ImageDataGenerator(rescale=1 / 255, validation_split=0.10)

#
test_datagen = ImageDataGenerator(rescale=1 / 255)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(dataframe=train_df,
                                                    x_col="Path",
                                                    y_col="ClassId",
                                                    target_size=image_size,
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    classes=l_classes,
                                                    subset='training',
                                                    shuffle=True,
                                                    seed=42)

val_generator = train_datagen.flow_from_dataframe(dataframe=train_df,
                                                  x_col="Path",
                                                  y_col="ClassId",
                                                  target_size=image_size,
                                                  batch_size=batch_size,
                                                  class_mode='categorical',
                                                  classes=l_classes,
                                                  subset='validation',
                                                  shuffle=True,
                                                  seed=42)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col="Path",
    y_col="ClassId",
    target_size=image_size,
    batch_size=1,
    class_mode=None,
    shuffle=False,
)

## Custom model

In [ ]:
model = Sequential()

model.add(
    layers.Conv2D(32, (3, 3),
                  activation='relu',
                  input_shape=(width_im, height_im, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(nb_classes, activation='softmax'))

print(model.summary())

In [ ]:
def plot_acc_Train_Val(history):
    fig, ax = plt.subplots(1, 2, figsize=(18, 6))
    fig.subplots_adjust(left=0.02, right=0.98, wspace=0.2)

    plt.rcParams.update({'font.size': 10})

    # Plot training & validation accuracy values
    ax[0].plot(history.history['accuracy'], label='Training')
    ax[0].plot(history.history['val_accuracy'], label='Validation')
    ax[0].set_title('Model accuracy')
    ax[0].set_ylabel('Accuracy')
    ax[0].set_xlabel('Epoch')
    ax[0].legend()

    # Plot training & validation loss values
    ax[1].plot(history.history['loss'], label='Training')
    ax[1].plot(history.history['val_loss'], label='Validation')
    ax[1].set_title('Model loss')
    ax[1].set_ylabel('Loss')
    ax[1].set_xlabel('Epoch')
    ax[1].legend()

    plt.show()

In [ ]:
def plot_cm_report(y_pred_class):

    cm = confusion_matrix(test_df['ClassId'], y_pred_class, labels=l_classes)
    disp = ConfusionMatrixDisplay(cm, display_labels=l_classes)
    fig, ax = plt.subplots(figsize=(15, 15))
    disp.plot(ax=ax)
    plt.title(f'Confusion matrix')
    plt.xticks(rotation=90)
    plt.show()

    print(classification_report(test_df['ClassId'], y_pred_class))

In [ ]:
def plot_25im_prdct(y_pred_class):
    plt.figure(figsize=(15, 15))
    rndm_idx = np.random.randint(0, test_df.shape[0], 25)
    for i, image_index in enumerate(rndm_idx):
        plt.subplot(5, 5, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(plt.imread(test_df.loc[image_index, 'Path']))
        plt.xlabel(test_df.loc[image_index, 'ClassId'])
        plt.title(y_pred_class[image_index])
    plt.show()

In [ ]:
def compile_fit_predict_plot(model, plotcm=True, plot_sample=False):

    model.compile(loss='categorical_crossentropy',
                  optimizer="adam",
                  metrics=['accuracy'])

    history = model.fit(train_generator,
                        epochs=2,
                        validation_data=val_generator)

    plot_acc_Train_Val(history)

    print('Predictions')
    predict = model.predict(test_generator)
    y_pred_class = predict.argmax(axis=-1)
    y_pred_class = [classes[x] for x in y_pred_class]

    if plot_cm_report: plot_cm_report(y_pred_class)

    if plot_sample: plot_25im_prdct(y_pred_class)

In [ ]:
compile_fit_predict_plot(model)

## Pretrained Model

### ResNet50

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

In [ ]:
model = ResNet50(input_shape=(width_im,height_im,3),weights=None,classes=nb_classes)

#fit


img_path = test_df.iloc[0,3]
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = model.predict(x)

# decode the results into a list of tuples (class, description, probability)
print('Predicted:', decode_predictions(preds, top=3)[0])

In [ ]:
model=VGG16(input_shape=(width_im,height_im,3),weights=None,classes=nb_classes)

In [ ]:
print(model.summary())

In [ ]:
compile_fit_predict_plot(model)